# Chains in LangChain

Watch:
https://learn.deeplearning.ai/langchain/lesson/4/chains 

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ : bool = load_dotenv(find_dotenv()) # read local .env file

In [6]:
llm_model: str = "gpt-3.5-turbo-0613"

In [2]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [3]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [7]:
llm: ChatOpenAI = ChatOpenAI(temperature=0.9, model=llm_model)

In [8]:
prompt: ChatPromptTemplate = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [9]:
chain: LLMChain = LLMChain(llm=llm, prompt=prompt)

In [11]:
product: str = "Queen Size Sheet Set"
response: str = chain.run(product)

'"RoyalRest Bedding" would be a suitable name to describe a company that makes Queen Size Sheet Sets.'